In [2]:
#import pymorton
from numba import jit

In [2]:
x = 1200
print(bin(x))
for i in range(1, 16):
    print((x>>i) & 0b000000000001)

0b10010110000
0
0
0
1
1
0
1
0
0
1
0
0
0
0
0


In [3]:
@jit(nopython=True)
def morton_encode3d(x, y, z):
    answer = 0;
    for i in range(1,21):
        answer |= ((x & (1 << i)) << 2*i) | ((y & (1 << i)) << (2*i + 1)) | ((z & (1 << i)) << (2*i + 2));
    return answer

@jit(nopython=True)
def morton_encode2d(x, y):
    answer = 0
    for i in range(0, 40):
        answer |= ((x & (1 << i)) << i) | ((y & (1 << i)) << (i + 1))
    return answer

@jit(nopython=True)
def morton_decode2d(m):
    x = 0
    y = 0
    for i in range(0, 22):
        selector = 1
        shift_selector = 2 * i
        x |= (m & (selector << shift_selector)) >> i
        y |= (m & (selector << (shift_selector + 1))) >> (i + 1)
    return x, y

@jit(nopython=True)
def morton_decode3d(m): 
    x = 0
    y = 0
    z = 0
    for i in range(0, 22):
        selector = 1;
        shift_selector = 3 * i
        shiftback = 2 * i
        x |= (m & (selector << shift_selector)) >> (shiftback)
        y |= (m & (selector << (shift_selector + 1))) >> (shiftback + 1)
        z |= (m & (selector << (shift_selector + 2))) >> (shiftback + 2)
    return x, y, z

In [4]:
morton_encode3d(10,10,10)

3640

In [85]:
m = morton2dEncode(2097151,2097151)

In [87]:
%timeit morton_decode2d(m)

278 ns ± 20 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [88]:
%timeit morton_encode2d(2097151,2097151)

221 ns ± 16.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [80]:
m = morton2dEncode(2097151,2097151)
morton_decode2d(m)

(2097151, 2097151)

In [ ]:
%timeit mortonEncode_for(1200,10,10)

In [ ]:
x = 1200
y = 10
z = 10
m = mortonEncode_for(x, y, z)
print("Morton code: ", m)
print("Decoded: ", morton_decode(m))

In [ ]:
#// method to seperate bits from a given integer 3 positions apart
def splitBy3(a):
    x = a & 0x1fffff; #// we only look at the first 21 bits
    x = (x | x << 32) & 0x1f00000000ffff; #// shift left 32 bits, OR with self, and 00011111000000000000000000000000000000001111111111111111
    x = (x | x << 16) & 0x1f0000ff0000ff; #// shift left 32 bits, OR with self, and 00011111000000000000000011111111000000000000000011111111
    x = (x | x << 8) & 0x100f00f00f00f00f; #// shift left 32 bits, OR with self, and 0001000000001111000000001111000000001111000000001111000000000000
    x = (x | x << 4) & 0x10c30c30c30c30c3; #// shift left 32 bits, OR with self, and 0001000011000011000011000011000011000011000011000011000100000000
    x = (x | x << 2) & 0x1249249249249249;
    return x;

def mortonEncode_magicbits(x, y, z):
    answer = 0;
    answer |= splitBy3(x) | splitBy3(y) << 1 | splitBy3(z) << 2;
    return answer

In [5]:
@jit(nopython=True)
def mortonEncode_magicbits(x, y, z):
    answer = 0
    
    x = x & 0x1fffff; 
    x = (x | x << 32) & 0x1f00000000ffff 
    x = (x | x << 16) & 0x1f0000ff0000ff 
    x = (x | x << 8) & 0x100f00f00f00f00f
    x = (x | x << 4) & 0x10c30c30c30c30c3
    x = (x | x << 2) & 0x1249249249249249
    
    y = y & 0x1fffff; 
    y = (y | y << 32) & 0x1f00000000ffff 
    y = (y | y << 16) & 0x1f0000ff0000ff 
    y = (y | y << 8) & 0x100f00f00f00f00f
    y = (y | y << 4) & 0x10c30c30c30c30c3
    y = (y | y << 2) & 0x1249249249249249
    
    z = z & 0x1fffff; 
    z = (z | z << 32) & 0x1f00000000ffff 
    z = (z | z << 16) & 0x1f0000ff0000ff 
    z = (z | z << 8) & 0x100f00f00f00f00f
    z = (z | z << 4) & 0x10c30c30c30c30c3
    z = (z | z << 2) & 0x1249249249249249
    
    answer |= x | y << 1 | z << 2;
    return answer

In [6]:
m = mortonEncode_magicbits(1500,10,10)
m

1092886128

In [ ]:
1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 1 2 3
z y x z y x z y x z y z z y x z y x z y x
  1     2     3     4     5     6     7

In [ ]:
x3_mask = 0x9249249249249249
y3_mask = 0x2492492492492492
z3_mask = 0x4924924924924924
xy3_mask = x3_mask | y3_mask
xz3_mask = x3_mask | z3_mask
yz3_mask = y3_mask | z3_mask

zx3_mask = z3_mask | x3_mask
yx3_mask = y3_mask | x3_mask
zy3_mask = z3_mask | y3_mask


x2_mask = 0x55555555
y2_mask = 0xAAAAAAAA

print(hex(x_mask))
print(hex(y_mask))
print(hex(z_mask))

In [22]:
#/* Add two morton keys (xy interleaving) */
def Morton2Add(m1, m2):
    x_sum = ( m1 | y2_mask) + (m2 & x2_mask)
    y_sum = ( m1 | x2_mask) + (m2 & y2_mask)
    return (x_sum & x2_mask) | (y_sum & y2_mask)

#/* Add two morton keys (xy interleaving) */
def Morton2Sub(m1, m2):
    x_sum = ( m1 & x2_mask) - (m2 & x2_mask)
    y_sum = ( m1 & y2_mask) - (m2 & y2_mask)
    return (x_sum & x2_mask) | (y_sum & y2_mask)

#/* Add two morton keys (xyz interleaving) */
def Morton3Add(m1, m2):
    x_sum = ( m1 | yz3_mask) + (m2 & x3_mask)
    y_sum = ( m1 | xz3_mask) + (m2 & y3_mask)
    z_sum = ( m1 | xy3_mask) + (m2 & z3_mask)
    return (x_sum & x3_mask ) | (y_sum & y3_mask ) | (z_sum & z3_mask)

#/* Add two morton keys (xyz interleaving) */
def Morton3Sub(m1, m2):
    x_sum = ( m1 & x3_mask) - (m2 & x3_mask)
    y_sum = ( m1 & y3_mask) - (m2 & y3_mask)
    z_sum = ( m1 & z3_mask) - (m2 & z3_mask)
    return (x_sum & x3_mask ) | (y_sum & y3_mask ) | (z_sum & z3_mask)

#@jit(nopython=True)
def mortonEncode_magicbits(x, y, z):
    answer = 0
    
    x = x & 0x1fffff; 
    x = (x | x << 32) & 0x1f00000000ffff 
    x = (x | x << 16) & 0x1f0000ff0000ff 
    x = (x | x << 8) & 0x100f00f00f00f00f
    x = (x | x << 4) & 0x10c30c30c30c30c3
    x = (x | x << 2) & 0x1249249249249249
    
    y = y & 0x1fffff; 
    y = (y | y << 32) & 0x1f00000000ffff 
    y = (y | y << 16) & 0x1f0000ff0000ff 
    y = (y | y << 8) & 0x100f00f00f00f00f
    y = (y | y << 4) & 0x10c30c30c30c30c3
    y = (y | y << 2) & 0x1249249249249249
    
    z = z & 0x1fffff; 
    z = (z | z << 32) & 0x1f00000000ffff 
    z = (z | z << 16) & 0x1f0000ff0000ff 
    z = (z | z << 8) & 0x100f00f00f00f00f
    z = (z | z << 4) & 0x10c30c30c30c30c3
    z = (z | z << 2) & 0x1249249249249249
    
    answer |= x | y << 1 | z << 2;
    return answer



In [33]:
# max size for a 64bit int is 3*21 bit number (63 bits).
# 2^21 = 2097152 -1 is the max size for each axis at 21 bits

m = mortonEncode_magicbits(2097151, 2097151, 2097151)
print(m)
print(bin(m))
morton_decode3d(m)

9223372036854775807
0b111111111111111111111111111111111111111111111111111111111111111


(2097151, 2097151, 2097151)

In [ ]:
m1 = mortonEncode_magicbits(80,100,50)
m2 = mortonEncode_magicbits(5, 2, 5)

morton_decode(Morton3Sub(m1, m2))

In [ ]:
a = pymorton.interleave(10,10)
b = pymorton.interleave(5,5)